# Variaciones Information Value

En esta notebook vemos las distintas variantes de Information Value y cuánto laburo extra demandarían

In [1]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("../data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["candidata"] = analizadas["Palabra Candidata"]

In [2]:
palabras_ya_analizadas = analizadas[analizadas['candidata'].notna()]

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [3]:
import re

df = pd.read_csv("../output/provinces_words.csv", index_col=0)

cant_palabras = [c for c in df.columns if re.match(r'.*ocurrencias$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*usuarios$', c)]

print("columnas de palabras => {}\n".format(cant_palabras))
print("columnas de usuarios => {}".format(cant_personas))

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)
df["cant_palabra"] = df[cant_palabras].sum(axis=1)
df["cant_usuarios"] = df[cant_personas].sum(axis=1)

columnas de palabras => ['buenosaires_ocurrencias', 'catamarca_ocurrencias', 'chaco_ocurrencias', 'chubut_ocurrencias', 'cordoba_ocurrencias', 'corrientes_ocurrencias', 'entrerios_ocurrencias', 'formosa_ocurrencias', 'jujuy_ocurrencias', 'lapampa_ocurrencias', 'larioja_ocurrencias', 'mendoza_ocurrencias', 'misiones_ocurrencias', 'neuquen_ocurrencias', 'rionegro_ocurrencias', 'salta_ocurrencias', 'sanjuan_ocurrencias', 'sanluis_ocurrencias', 'santacruz_ocurrencias', 'santafe_ocurrencias', 'santiago_ocurrencias', 'tierradelfuego_ocurrencias', 'tucuman_ocurrencias']

columnas de usuarios => ['buenosaires_usuarios', 'catamarca_usuarios', 'chaco_usuarios', 'chubut_usuarios', 'cordoba_usuarios', 'corrientes_usuarios', 'entrerios_usuarios', 'formosa_usuarios', 'jujuy_usuarios', 'lapampa_usuarios', 'larioja_usuarios', 'mendoza_usuarios', 'misiones_usuarios', 'neuquen_usuarios', 'rionegro_usuarios', 'salta_usuarios', 'sanjuan_usuarios', 'sanluis_usuarios', 'santacruz_usuarios', 'santafe_usuario

In [4]:
df = df[(df.cant_palabra >= 100) & (df.cant_usuarios >= 25) ]

Calculemos el valor de la información de cada palabra.

In [5]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log(23) - df["entropy_palabras"]
df["delta_personas"] = np.log(23) - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)


Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [6]:
def calcular_porcentaje_candidatas(palabras):
    subconj_analizado = palabras.index.intersection(palabras_ya_analizadas.index)
    candidatas = (analizadas.loc[subconj_analizado]['candidata'] != '0').sum()
    total = subconj_analizado.shape[0]
    print("Palabras candidatas = {}".format(candidatas))
    print("Total analizadas actualmente = {}".format(total))
    
    return candidatas / total

def analizar_palabras_restantes(n, dataframes):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    
    for nombre, this_df in dataframes.items(): 
        print('#'*40)
        print("Métrica: {}".format(nombre))
        palabras = this_df.iloc[:n]
        
        metrica = calcular_porcentaje_candidatas(palabras)
        restantes = {p for p in palabras.index if p not in palabras_ya_analizadas.index}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("Faltan etiquetar {} palabras".format(len(restantes)))
        print("Porcentaje candidatas = {}".format(metrica))
    print("Faltan etiquetar (sin repeticiones) = {}".format(len(todas_las_palabras)))

In [7]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp,
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 176
Total analizadas actualmente = 787
Faltan etiquetar 213 palabras
Porcentaje candidatas = 0.22363405336721728
########################################
Métrica: Personas
Palabras candidatas = 209
Total analizadas actualmente = 749
Faltan etiquetar 251 palabras
Porcentaje candidatas = 0.27903871829105475
########################################
Métrica: PalPer
Palabras candidatas = 192
Total analizadas actualmente = 780
Faltan etiquetar 220 palabras
Porcentaje candidatas = 0.24615384615384617
Faltan etiquetar (sin repeticiones) = 346
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1444
Total analizadas actualmente = 3398
Faltan etiquetar 1602 palabras
Porcentaje candidatas = 0.4249558563861095
########################################
Métrica: PalPer
Palabras candidatas = 1357
Total analizadas actualmente = 3341
Faltan etiquetar

## Sin usar el log(23)

In [8]:
from contrastes.information_value import shuffled_entropy
np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas
df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [9]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 180
Total analizadas actualmente = 796
Faltan etiquetar 204 palabras
Porcentaje candidatas = 0.22613065326633167
########################################
Métrica: Personas
Palabras candidatas = 229
Total analizadas actualmente = 773
Faltan etiquetar 227 palabras
Porcentaje candidatas = 0.296248382923674
########################################
Métrica: PalPer
Palabras candidatas = 206
Total analizadas actualmente = 798
Faltan etiquetar 202 palabras
Porcentaje candidatas = 0.2581453634085213
Faltan etiquetar (sin repeticiones) = 342
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1524
Total analizadas actualmente = 3476
Faltan etiquetar 1524 palabras
Porcentaje candidatas = 0.43843498273878023
########################################
Métrica: PalPer
Palabras candidatas = 1420
Total analizadas actualmente = 3434
Faltan etiquetar 1

## Usando cálculo con normalización "vieja"


In [10]:
np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)


In [11]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [12]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 265
Total analizadas actualmente = 827
Faltan etiquetar 173 palabras
Porcentaje candidatas = 0.32043530834340994
########################################
Métrica: Personas
Palabras candidatas = 367
Total analizadas actualmente = 900
Faltan etiquetar 100 palabras
Porcentaje candidatas = 0.4077777777777778
########################################
Métrica: PalPer
Palabras candidatas = 360
Total analizadas actualmente = 968
Faltan etiquetar 32 palabras
Porcentaje candidatas = 0.371900826446281
Faltan etiquetar (sin repeticiones) = 262
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1707
Total analizadas actualmente = 3509
Faltan etiquetar 1491 palabras
Porcentaje candidatas = 0.4864633798803078
########################################
Métrica: PalPer
Palabras candidatas = 1608
Total analizadas actualmente = 3444
Faltan etiquetar 155

## Usando el cálculo viejo CASI idéntico

No usó normalización en las dos primeras métricas

In [13]:

np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))


df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)
df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)

In [14]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.log_cantidad * df.delta_palabras
df["ival_personas"] = df.log_personas * df.delta_personas
df["ival"] = df.norm_cantidad * df.norm_personas * df.delta_palabras * df.delta_personas

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [15]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 179
Total analizadas actualmente = 795
Faltan etiquetar 205 palabras
Porcentaje candidatas = 0.22515723270440252
########################################
Métrica: Personas
Palabras candidatas = 228
Total analizadas actualmente = 777
Faltan etiquetar 223 palabras
Porcentaje candidatas = 0.29343629343629346
########################################
Métrica: PalPer
Palabras candidatas = 360
Total analizadas actualmente = 969
Faltan etiquetar 31 palabras
Porcentaje candidatas = 0.3715170278637771
Faltan etiquetar (sin repeticiones) = 342
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1528
Total analizadas actualmente = 3472
Faltan etiquetar 1528 palabras
Porcentaje candidatas = 0.4400921658986175
########################################
Métrica: PalPer
Palabras candidatas = 1615
Total analizadas actualmente = 3452
Faltan etiquetar 1

## Conclusión

Usaremos la métrica "más linda", la primera: con el log(23) y la normalización más bestia.